In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import autorootcwd
import os
import pandas as pd
import json

## LR Scheduler Search

In [22]:
DIR = 'results/crnn_lr_search'
experiments = os.listdir(DIR)

rows = []
errored = []

for experiment in experiments:
    with open(f'{DIR}/{experiment}/training_args.json') as f:
        args = json.load(f)
    lr = args['lr']
    scheduler = args['lr_scheduler']

    with open(f'{DIR}/{experiment}/val_metrics.json') as f:
        data = json.load(f)
    rows.append({
        'lr': lr,
        'scheduler': scheduler,
        'accuracy': data['mean']['acc'],
        'root': data['mean']['root'],
        'majmin': data['mean']['majmin'],
        'third': data['mean']['third'],
        'seventh': data['mean']['seventh'],
        'mirex': data['mean']['mirex'],
        'class_wise_acc_mean': data['class_wise']['acc']['mean'],
        'class_wise_acc_median': data['class_wise']['acc']['median'],
    })

df = pd.DataFrame(rows)

In [25]:
df

,lr,scheduler,accuracy,root,majmin,third,seventh,mirex,class_wise_acc_mean,class_wise_acc_median
8,0.00001,cosine,7.734154,12.567862,10.318443,10.301985,7.948638,11.407809,0.700043,0.000000
9,0.00001,none,11.969150,17.934697,15.201270,15.129656,12.358253,16.698056,1.142158,0.000000
12,0.00001,plateau,11.969150,17.934697,15.201270,15.129656,12.358253,16.698056,1.142158,0.000000
13,0.00010,cosine,53.176047,72.122734,67.605339,66.869413,55.225218,72.039027,10.874403,0.000000
3,0.00010,none,56.036913,75.729819,72.177384,71.370852,58.233276,76.150540,13.223820,0.000000
5,0.00010,plateau,56.036913,75.729819,72.177384,71.370852,58.233276,76.150540,13.223820,0.000000
0,0.00100,cosine,59.693026,78.289060,75.692472,75.038317,61.967582,79.788345,19.589127,2.329573
6,0.00100,none,59.757152,78.696339,76.156221,75.476839,62.032520,78.768536,18.378774,0.673294
2,0.00100,plateau,59.886951,78.385394,75.878873,75.198397,62.157333,79.680444,19.543125,2.329573
1,0.01000,cosine,53.612974,69.548986,67.492188,66.851165,55.681087,78.561550,13.680832,0.000000


In [24]:
df.sort_values(by=['lr', 'scheduler'], inplace=True)

In [19]:
df[(df['scheduler'] == 'cosine') | (df['lr'] == '0.001')]

,lr,scheduler,accuracy,root,majmin,third,seventh,mirex,class_wise_acc_mean,class_wise_acc_median
0,0.00100,cosine,59.693026,78.289060,75.692472,75.038317,61.967582,79.788345,19.589127,19.589127
1,0.01000,cosine,53.612974,69.548986,67.492188,66.851165,55.681087,78.561550,13.680832,13.680832
4,0.10000,cosine,7.093379,12.179759,9.062168,8.932016,7.338634,9.852282,0.699301,0.699301
8,0.00001,cosine,7.734154,12.567862,10.318443,10.301985,7.948638,11.407809,0.700043,0.700043
13,0.00010,cosine,53.176047,72.122734,67.605339,66.869413,55.225218,72.039027,10.874403,10.874403


# Random HParam Search Results

In [51]:
experiments = os.listdir('results/crnn_hparams')
# Make sure its a dir
experiments = [e for e in experiments if os.path.isdir(f'results/crnn_hparams/{e}')]

rows = []
errored = []

for experiment in experiments:
    # Format of filename is segment_length_N_layers_N_hidden_size_N
    with open(f'results/crnn_hparams/{experiment}/metadata.json') as f:
        args = json.load(f)

    kernel_size = args['args']['cnn_kernel_size']
    cnn_layers = args['args']['cnn_layers']
    gru_layers = args['args']['gru_layers']
    cnn_channels = args['args']['cnn_channels']
    segment_length = args['args']['segment_length']
    cnn_layers = args['args']['cnn_layers']
    hidden_size = args['args']['hidden_size']
    try:
        with open(f'results/crnn_hparams/{experiment}/val_metrics.json') as f:
            data = json.load(f)
        rows.append({
            'segment_length': segment_length, 
            'hidden_size': hidden_size, 
            'kernel_size': kernel_size,
            'cnn_layers': cnn_layers,
            'gru_layers': gru_layers,
            'cnn_channels': cnn_channels,
            'accuracy': data['mean']['acc'],
            'root': data['mean']['root'],
            'majmin': data['mean']['majmin'],
            'third': data['mean']['third'],
            'seventh': data['mean']['seventh'],
            'mirex': data['mean']['mirex'],
            'class_wise_acc_mean': data['class_wise']['acc']['mean'],
            'class_wise_acc_median': data['class_wise']['acc']['median'],
        })
    except:
        continue

df = pd.DataFrame(rows)

In [52]:
df

,segment_length,hidden_size,kernel_size,cnn_layers,gru_layers,cnn_channels,accuracy,root,majmin,third,seventh,mirex,class_wise_acc_mean,class_wise_acc_median
0,37,282,8,4,1,4,59.051987,77.659056,75.083176,74.430356,61.334676,79.086615,19.710527,2.045311
1,40,255,9,3,1,3,58.875829,77.591812,74.520398,73.942881,61.092743,79.628474,19.091770,1.898007
2,37,355,13,1,1,1,58.842825,77.631947,74.723874,73.965396,61.068292,77.956086,17.234292,0.000000
3,19,392,6,3,2,2,58.694414,77.542539,74.795254,74.225846,60.878814,79.310407,19.171167,1.448819
4,10,85,11,3,2,2,57.941735,77.266494,74.457087,73.779267,60.182881,80.712868,16.706184,0.000000
5,36,164,5,1,2,2,58.936852,77.417804,74.810801,74.086295,61.114534,80.479867,19.454971,0.000000
6,19,163,5,3,1,4,58.011021,77.642754,74.037520,73.392950,60.244543,80.211942,16.685454,0.000000
7,18,298,9,2,1,1,58.403208,77.833262,74.844866,74.236587,60.657659,78.187374,18.414961,0.384940
8,32,178,14,4,1,2,57.417350,76.544891,73.455567,72.824881,59.732062,79.602065,16.428447,0.000000
9,38,413,11,2,2,4,58.908086,78.677848,75.446905,74.808022,61.140663,78.547254,18.431776,3.458247


Chosen model: 28 segfment length, 1 layer, 201.

In [54]:
# Ranks
df['rank_accuracy'] = df['accuracy'].rank(ascending=False)
df['rank_root'] = df['root'].rank(ascending=False)
df['rank_majmin'] = df['majmin'].rank(ascending=False)
df['rank_third'] = df['third'].rank(ascending=False)
df['rank_seventh'] = df['seventh'].rank(ascending=False)
df['rank_mirex'] = df['mirex'].rank(ascending=False)

df['rank_sum'] = df['rank_accuracy'] + df['rank_root'] + df['rank_majmin'] + df['rank_third'] + df['rank_seventh'] + df['rank_mirex']
df['rank_sum'] = df['rank_sum'].rank(ascending=True)

df = df.sort_values(['rank_sum', 'root'], ascending=[True, False])
df

,segment_length,hidden_size,kernel_size,cnn_layers,gru_layers,cnn_channels,accuracy,root,majmin,third,...,mirex,class_wise_acc_mean,class_wise_acc_median,rank_accuracy,rank_root,rank_majmin,rank_third,rank_seventh,rank_mirex,rank_sum
26,23,321,5,1,1,1,59.835087,78.209675,75.298987,74.711638,...,79.855935,19.697356,2.527502,1.0,8.0,8.0,7.0,1.0,6.0,1.0
40,11,150,7,2,1,3,59.627560,78.526063,75.621006,74.996513,...,79.183307,20.064784,3.725503,3.0,4.0,2.0,3.0,3.0,20.0,2.0
28,43,222,11,2,2,2,59.541313,78.735225,75.825075,75.165983,...,78.954202,19.264325,1.143783,4.0,2.0,1.0,1.0,4.0,24.0,3.0
25,22,239,10,1,2,1,59.297003,78.877479,75.600548,75.041850,...,79.164656,19.295269,3.640895,7.0,1.0,3.0,2.0,8.0,21.0,4.0
33,9,435,5,2,1,2,59.782710,78.118600,75.084287,74.549427,...,79.637113,20.416666,5.375451,2.0,11.0,15.0,13.0,2.0,9.0,5.0
19,25,67,9,1,2,2,58.929073,78.478943,75.401979,74.706412,...,79.829044,19.205363,0.000000,16.0,5.0,7.0,8.0,15.0,7.0,6.0
29,25,77,10,3,1,4,59.325375,78.473305,75.473141,74.895228,...,78.380004,18.022180,0.232937,5.0,6.0,5.0,5.0,5.0,35.0,7.0
37,22,310,7,1,1,2,59.285266,78.273048,75.501423,74.915418,...,78.641844,19.363420,3.518361,8.0,7.0,4.0,4.0,9.0,31.0,8.0
21,32,147,8,4,1,3,59.251492,78.190898,75.291960,74.676544,...,78.302584,18.304943,0.618686,9.0,9.0,9.0,9.0,7.0,36.0,9.0
9,38,413,11,2,2,4,58.908086,78.677848,75.446905,74.808022,...,78.547254,18.431776,3.458247,18.0,3.0,6.0,6.0,16.0,32.0,10.0


In [45]:
df.iloc[[0,1,2,18,19]]

,segment_length,hidden_size,kernel_size,cnn_layers,gru_layers,cnn_channels,accuracy,root,majmin,third,...,mirex,class_wise_acc_mean,class_wise_acc_median,rank_accuracy,rank_root,rank_majmin,rank_third,rank_seventh,rank_mirex,rank_sum
1,13,150,6,1,3,4,60.143913,79.434660,76.792206,76.088499,...,80.707666,19.449476,0.000000,3.0,1.0,1.0,2.0,3.0,5.0,1.0
7,9,242,7,1,3,4,60.919885,79.000992,76.792172,76.153159,...,79.876940,19.749170,0.809089,1.0,3.0,2.0,1.0,1.0,9.0,2.0
4,8,195,6,1,3,2,60.362169,79.144172,76.789880,76.081944,...,80.530634,19.030529,0.608992,2.0,2.0,3.0,3.0,2.0,7.0,3.0
10,16,44,7,3,1,2,52.036654,71.780499,67.107501,66.473048,...,83.975498,12.207555,0.000000,20.0,20.0,20.0,20.0,20.0,1.0,18.5
16,29,41,7,1,1,1,56.673692,76.490359,73.103443,72.347434,...,76.114278,13.945593,0.000000,18.0,18.0,18.0,18.0,18.0,20.0,20.0


In [8]:
df['avg_score'] = df[['accuracy', 'root', 'majmin', 'third', 'seventh', 'mirex']].mean(axis=1)
df = df.sort_values('avg_score', ascending=False)
df

,segment_length,layers,hidden_size,accuracy,root,majmin,third,seventh,mirex,rank_accuracy,rank_root,rank_majmin,rank_third,rank_seventh,rank_mirex,rank_sum,avg_score
3,15,3,365,0.581273,0.783976,0.761268,0.753708,0.621261,0.786819,3.0,2.0,1.0,1.0,1.0,24.0,1.0,0.714718
26,28,1,201,0.584848,0.783438,0.751750,0.745390,0.615702,0.791808,1.0,3.0,7.0,6.0,5.0,16.0,2.0,0.712156
30,17,3,104,0.578276,0.780257,0.755824,0.748137,0.616489,0.792109,7.0,6.0,4.0,4.0,3.0,15.0,3.0,0.711849
8,23,2,295,0.579131,0.781325,0.757069,0.750643,0.617953,0.783877,5.0,4.0,3.0,3.0,2.0,27.0,4.0,0.711666
16,28,3,258,0.572051,0.785424,0.758784,0.752242,0.612319,0.783116,15.0,1.0,2.0,2.0,8.0,29.0,7.0,0.710656
10,14,3,405,0.578178,0.780909,0.752852,0.746195,0.614306,0.790389,8.0,5.0,5.0,5.0,6.0,20.0,5.0,0.710471
18,41,2,109,0.575577,0.775753,0.752236,0.744862,0.610782,0.799764,11.0,11.0,6.0,7.0,9.0,6.0,6.0,0.709829
25,11,1,306,0.583275,0.775839,0.750888,0.743953,0.616366,0.786460,2.0,10.0,10.0,9.0,4.0,25.0,8.0,0.709463
2,34,2,117,0.580755,0.775544,0.747836,0.740625,0.610437,0.801393,4.0,12.0,15.0,15.0,11.0,5.0,9.0,0.709432
1,23,2,218,0.578347,0.770169,0.748675,0.742194,0.612861,0.795408,6.0,22.0,13.0,13.0,7.0,11.0,10.0,0.707943


# Hyperparameter Results

In [ ]:
rows = []

for segment_length in [10,20,30,60]:
    for n_layers in [1,2,3]:
        for hidden_size in [128,256,512]:
            try:
                with open(f"results/hparam_search/segment_length_{segment_length}_layers_{n_layers}_hidden_size_{hidden_size}/val_metrics.json") as f:
                    data = json.load(f)
                    rows.append({
                        'segment_length': segment_length,
                        'n_layers': n_layers,
                        'hidden_size': hidden_size,
                        'accuracy': data['mean']['song_wise_acc'],
                        'root': data['mean']['root'],
                        'majmin': data['mean']['majmin'],
                        'third': data['mean']['third'],
                        'seventh': data['mean']['seventh'],
                        'mirex': data['mean']['mirex'],
                        'class_wise_acc_mean': data['class_wise_acc_mean'],
                        'class_wise_acc_median': data['class_wise_acc_median'],
                    })
            except:
                print(f"results/hparam_search/segment_length_{segment_length}_layers_{n_layers}_hidden_size_{hidden_size}/val_metrics.json not found")
                pass

df = pd.DataFrame(rows)

In [40]:
# Ranks
df['rank_accuracy'] = df['accuracy'].rank(ascending=False)
df['rank_root'] = df['root'].rank(ascending=False)
df['rank_majmin'] = df['majmin'].rank(ascending=False)
df['rank_third'] = df['third'].rank(ascending=False)
df['rank_seventh'] = df['seventh'].rank(ascending=False)
df['rank_mirex'] = df['mirex'].rank(ascending=False)

df['rank_sum'] = df['rank_accuracy'] + df['rank_root'] + df['rank_majmin'] + df['rank_third'] + df['rank_seventh'] + df['rank_mirex']
df['rank_sum'] = df['rank_sum'].rank(ascending=True)

df = df.sort_values('rank_sum')
df

,segment_length,n_layers,hidden_size,accuracy,root,majmin,third,seventh,mirex,class_wise_acc_mean,class_wise_acc_median,rank_accuracy,rank_root,rank_majmin,rank_third,rank_seventh,rank_mirex,rank_sum
6,10,3,128,0.582922,0.790673,0.762577,0.755801,0.619829,0.788412,0.185961,0.014381,1.0,1.0,1.0,1.0,1.0,20.0,1.0
27,60,1,128,0.577344,0.778589,0.753823,0.746398,0.612000,0.789618,0.184662,0.017244,4.0,7.0,6.0,6.0,6.0,17.0,2.0
28,60,1,256,0.579214,0.782663,0.755422,0.748741,0.617097,0.777011,0.185935,0.013484,2.0,3.0,4.0,4.0,2.0,33.0,3.5
34,60,3,256,0.576456,0.780509,0.755414,0.748729,0.615205,0.784500,0.185241,0.014624,5.0,5.0,5.0,5.0,3.0,25.0,3.5
30,60,2,128,0.574241,0.783396,0.757267,0.749927,0.612031,0.782075,0.173214,0.000000,10.0,2.0,2.0,2.0,5.0,28.0,5.0
18,30,1,128,0.579209,0.775584,0.750540,0.743974,0.614461,0.798464,0.189845,0.015776,3.0,15.0,12.0,12.0,4.0,10.0,6.0
26,30,3,512,0.574481,0.778038,0.751947,0.745281,0.611859,0.787995,0.174470,0.009105,9.0,9.0,8.0,8.0,7.0,21.0,7.0
16,20,3,256,0.574667,0.775172,0.752359,0.745658,0.610584,0.792820,0.181625,0.013714,8.0,16.0,7.0,7.0,9.0,16.0,8.0
31,60,2,256,0.575264,0.780037,0.750958,0.744152,0.610339,0.783360,0.175890,0.007937,6.0,6.0,11.0,11.0,10.0,26.0,9.0
13,20,2,256,0.571705,0.777338,0.751134,0.744851,0.607963,0.786980,0.181296,0.021021,13.0,11.0,10.0,9.0,14.0,22.0,10.0
